In [2]:
from google.cloud import bigquery as bq
import os

service_credentials = 'Service_Credentials/big-query-horse-play-f37757d450b8.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_credentials

In [4]:
# Instantiates bq client, optionally requires project
bq_client = bq.Client()

In [35]:
# Utility function for returning available datasets in project
def get_datasets(client_name):
    datasets = list(client_name.list_datasets())
    
    if datasets:
        print(f'Datasets in project - {client_name.project}:')
        for dataset in datasets:
            print(dataset.dataset_id)
    else:
        print(f'No datasets in {client_name.project}')
        
    return datasets

# Utility function for creating dataset
def create_dataset(client_name, dataset_id, dataset_location=None, project=None):
    
    # Creates dataset reference for bq
    dataset_reference = client_name.dataset(dataset_id, project)
    
    # Creates actual data set object
    dataset = bq.Dataset(dataset_reference)
    
    # Optionally sets dataset location value
    if dataset_location:
        dataset.location = dataset_location
    
    # Makes the call home
    dataset = client_name.create_dataset(dataset)
    
    return f'{dataset_id} created in project - {client_name.project}'